
**Навчання з використанням передвиборної мережі​**



Існує два основних підходи, які використовують попередньо навчені мережі:



- Виділення ознак (feature extraction).
- Донавчання (fine-tuning).

**Training with a Pretrained Network**

There are two main approaches to using pretrained networks:

- Feature Extraction

- Fine-Tuning

In [49]:
# # Check active kernel
# !jupyter kernelspec list

# # Install IPython kernel for your venv
# !pip install ipykernel
# !python -m ipykernel install --user --name=.venv --display-name="Python (.venv)"

In [50]:
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))


In [51]:
import tensorflow as tf

from keras.applications.vgg16 import VGG16
from keras import models
from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from keras import optimizers


conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = False

model = models.Sequential([
   conv_base,
   layers.Flatten(),
   layers.Dense(256, activation="relu"),
   layers.Dense(1, activation="sigmoid"),
])

model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    metrics=["acc"]
)


In [52]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,812,353 (64.13 MB)

 Trainable params: 2,097,665 (8.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [53]:
# train_dir = './dataset/train'
# validation_dir ='./dataset/validation'

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode="nearest"
# )
# test_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#     train_dir, 
#     target_size=(150, 150),
#     batch_size=20,
#     class_mode="binary"
# )

# validation_generator = test_datagen.flow_from_directory(
#     validation_dir,
#     target_size=(150, 150),
#     batch_size=20,
#     class_mode="binary"
# )


In [54]:
# from PIL import Image
# img = Image.open('./dataset/test/cats/cat.1500.jpg')
# img.show()


In [55]:
import PIL
print(PIL.__version__)

11.1.0


In [56]:
# history = model.fit(
#     train_generator,
#     steps_per_epoch=100,
#     epochs=100,
#     validation_data=validation_generator,
#     validation_steps=50
# )

Проаналізуємо графіки зміни точності та втрат на тестових та валідаційних даних:

Let's analyze the graphs of changes in accuracy and losses on test and validation data:

Навчання настільки великої мережі займає багато часу, тому є сенс зберегти її:

In [57]:
import os

current_dir = os.getcwd()

model_path = os.path.join(current_dir, "vgg16_based_model.hdf5")

# model.save(model_path)

In [58]:
model = load_model(model_path)
print("Модель успішно завантажено!")


Модель успішно завантажено!


In [59]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Приклад: завантаження зображення
img_path = "./dataset/validation/dogs/dog.1002.jpg"
img = image.load_img(img_path, target_size=(150, 150))

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # додати batch dimension
img_array = img_array / 255.0  # нормалізація (якщо потрібно)

# Передбачення
predictions = model.predict(img_array)
print("Результат передбачення:", predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Результат передбачення: [[0.99843115]]
